In [3]:
import numpy as np
import os
import pandas as pd
import pprint
import csv
import keras
#path constants
train_path = '../../../iterations/data/final/train_norm'
test_path = '../../../iterations/data/final/test_norm'

cleaned_train_path = '../../../cleaned_data/train'
cleaned_test_path = '../../../cleaned_data/test'
#type constants
vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']

#two label dataframes
train_label_df = pd.read_csv('../../../iterations/data/final' + '/label.csv', delimiter = ',', encoding = 'utf-8')
test_label_df = pd.read_csv('../../../iterations/data/final' + '/label.csv', delimiter = ',', encoding = 'utf-8')

Using TensorFlow backend.


In [4]:
def getLabel(filename, label_df):
    idx = label_df.loc[label_df['sample_file_name'] == filename]
    return idx.iloc[0]['label']

def cal_length(path, vehicle_type, label_df):

#vehicle_type: one string element under vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']
    path = path + '/' + vehicle_type
    #these are variables to calculate traversing progress (DO NOT CHANGE)
    counts_per_percent = int(len(os.listdir(path)) / 100)
    percentage_completion = 0
    counter = 0
    
    single_len=0
    file_count=0
    file_len=0
    for file in os.listdir(path):
        sample_df = pd.read_csv(path + '/' + file, delimiter = ',', encoding = 'utf-8')
        file_len+=len(sample_df)
        file_count+=1
        if len(sample_df)>=single_len:
            single_len=len(sample_df)
    ave_file_len=file_len/file_count
    print("AVG Length: ",ave_file_len,' file count: ',file_count,' max_len: ',single_len)

In [5]:
cal_length(train_path,'ZV252',train_label_df)

AVG Length:  200.85507246376812  file count:  345  max_len:  2961


In [41]:
file_len_weighted=round(392*0.7)
file_len_weighted

274

In [6]:
def getLabel(filename, label_df):
    idx = label_df.loc[label_df['sample_file_name'] == filename]
    return idx.iloc[0]['label']

def TraverseFiles(path, vehicle_type, label_df, length):

#vehicle_type: one string element under vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']
    path = path + '/' + vehicle_type
    #these are variables to calculate traversing progress (DO NOT CHANGE)
    counts_per_percent = int(len(os.listdir(path)) / 100)
    percentage_completion = 0
    counter = 0
    
    lables=np.array([])
    file_list= []
    #file_len=file_len_weighted
    for file in os.listdir(path):
        sample_df = pd.read_csv(path + '/' + file, delimiter = ',', encoding = 'utf-8')
        if len(sample_df)==0:
            continue
        elif len(sample_df)!=0:
            sample_array=np.array(sample_df.iloc[:,0:11])
            if len(sample_array)<length:
                n_zeros=length-len(sample_array)
                sample_array=np.concatenate((np.zeros((n_zeros,11)),sample_array))
                #print(len(sample_array))
            elif len(sample_array)>length:
                sample_array=sample_array[0:length]
                #print(len(sample_array))
            elif len(sample_array)==length:
                sample_array=sample_array
                #print(len(sample_array))
            #np.float(sample_array)
            #sample_array=torch.from_numpy(sample_array).float()
            #file_list=torch.cat((file_list,sample_array[0:76,]), dim=1)
        file_list.append(sample_array)
        l=np.array(label_df.loc[label_df['sample_file_name'] == file])
        lables=np.append(lables,[l[:,1]])
        #belows are to show traversing progress (DO NOT CHANGE)
        counter += 1
        if counter == counts_per_percent:
            counter = 0
            percentage_completion += 1
            print('traversing files under', path, ':', percentage_completion, "%", end="\r", flush=True)

    return file_list,lables

In [8]:
data_array_train, data_labels_train = TraverseFiles(train_path,'ZV252',train_label_df,210)
data_array_test, data_labels_test = TraverseFiles(test_path,'ZV252',test_label_df,210)

In [9]:
x_train=np.array(data_array_train)
labels_train=np.array(data_labels_train,dtype=int)

In [10]:
x_test=np.array(data_array_test)
labels_test=np.array(data_labels_test,dtype=int)

In [11]:
print(x_train.shape)
print(labels_train.shape)
print(x_test.shape)
print(labels_test.shape)

(345, 210, 11)
(345,)
(87, 210, 11)
(87,)


In [12]:
keras.backend.clear_session()
lstm = keras.Sequential()
lstm.add(keras.layers.Dropout(0.2))
lstm.add(keras.layers.LSTM(16,dropout=0.2, recurrent_dropout=0.2,return_sequences=True))
lstm.add(keras.layers.BatchNormalization())
lstm.add(keras.layers.LSTM(16,dropout=0.2, recurrent_dropout=0.2))
lstm.add(keras.layers.BatchNormalization())
lstm.add(keras.layers.Dense(1, activation = 'sigmoid'))

lstm.compile(
    loss='binary_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

Instructions for updating:
Colocations handled automatically by placer.


In [13]:
lstm.fit(x_train,
    labels_train,
    epochs = 500,
    batch_size=200
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/500
345/345 [==============================] - 2s 5ms/step - loss: 0.8379 - accuracy: 0.4783
Epoch 2/500
345/345 [==============================] - 0s 1ms/step - loss: 0.8342 - accuracy: 0.5014
Epoch 3/500
345/345 [==============================] - 0s 1ms/step - loss: 0.8056 - accuracy: 0.4957
Epoch 4/500
345/345 [==============================] - 0s 1ms/step - loss: 0.8227 - accuracy: 0.4812
Epoch 5/500
345/345 [==============================] - 0s 1ms/step - loss: 0.7345 - accuracy: 0.5565
Epoch 6/500
345/345 [==============================] - 0s 1ms/step - loss: 0.7544 - accuracy: 0.5449
Epoch 7/500
345/345 [==============================] - 0s 1ms/step - loss: 0.7811 - accuracy: 0.5159
Epoch 8/500
345/345 [==============================] - 0s 1ms/step - loss: 0.7677 - accuracy: 0.4986
Epoch 9/500
345/345 [==============================] - 0s 1ms/step - loss: 0.7444 - accuracy: 0.5594
Epoch 10/500
345/345 [=====================

345/345 [==============================] - 0s 1ms/step - loss: 0.6845 - accuracy: 0.5623
Epoch 80/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6637 - accuracy: 0.6319
Epoch 81/500
345/345 [==============================] - ETA: 0s - loss: 0.6135 - accuracy: 0.66 - 0s 1ms/step - loss: 0.6348 - accuracy: 0.6348
Epoch 82/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6659 - accuracy: 0.5913
Epoch 83/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6671 - accuracy: 0.5826
Epoch 84/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6769 - accuracy: 0.5623
Epoch 85/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6559 - accuracy: 0.6029
Epoch 86/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6446 - accuracy: 0.5884
Epoch 87/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6691 - accuracy: 0.6174
Epoch 88/500
345/345 [==============================]

345/345 [==============================] - 0s 1ms/step - loss: 0.6641 - accuracy: 0.6000
Epoch 238/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6468 - accuracy: 0.6261
Epoch 239/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6536 - accuracy: 0.6029
Epoch 240/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6624 - accuracy: 0.6000
Epoch 241/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6370 - accuracy: 0.6522
Epoch 242/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6405 - accuracy: 0.6348
Epoch 243/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6499 - accuracy: 0.6087
Epoch 244/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6631 - accuracy: 0.5942
Epoch 245/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6380 - accuracy: 0.6174
Epoch 246/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6594 - a

345/345 [==============================] - 0s 1ms/step - loss: 0.6352 - accuracy: 0.6522
Epoch 396/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6193 - accuracy: 0.6638
Epoch 397/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6334 - accuracy: 0.6609
Epoch 398/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6440 - accuracy: 0.6348
Epoch 399/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6343 - accuracy: 0.6435
Epoch 400/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6278 - accuracy: 0.6696
Epoch 401/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6316 - accuracy: 0.6464
Epoch 402/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6270 - accuracy: 0.6319
Epoch 403/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6497 - accuracy: 0.6145
Epoch 404/500
345/345 [==============================] - 0s 1ms/step - loss: 0.6442 - a

In [14]:
lstm.save('lstm_model_ZV252.h5')

In [15]:
pred_=lstm.predict_classes(x_test)

In [16]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print(classification_report(labels_test, pred_,digits=4))
print(accuracy_score(labels_test, pred_))

              precision    recall  f1-score   support

           0     0.7273    0.8163    0.7692        49
           1     0.7188    0.6053    0.6571        38

   micro avg     0.7241    0.7241    0.7241        87
   macro avg     0.7230    0.7108    0.7132        87
weighted avg     0.7236    0.7241    0.7203        87

0.7241379310344828
